# MPRA analysis of CMPRA data

In [1]:
import warnings
warnings.filterwarnings('ignore')
import polars as pl
import polars.selectors as cs
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
from scipy import stats
pl.Config.set_fmt_str_lengths(50)
warnings.filterwarnings('ignore')

## Read files

In [3]:
config = "promoteroabinaware"
cd = "/data/cephfs-2/unmirrored/groups/kircher/MPRA/CaptureCMPRA/"
file = cd+"mpra_capture_flow/results/CMPRA5/mpralm/"+config+"_OA_mpralm_output.tsv"
mpralm = pl.read_csv(file, separator="\t")
tss_file = cd+"resources/TSS_pos_v44.bed.gz"
alu_file = cd+"data/repeatMasker_annotations/hg38_2020_rmsk_Alus.bed.gz"
ccres_file=cd+"resources/GRCh38-cCREs.bed"
extended_ccres_file = cd+"resources/GRCh38-cCREs-extended.bed"
baits_file= cd+"resources/Arnould_3X_Mod_1_Covered.bed"
tss = pl.read_csv(cd+"resources/TSS_pos_v44.bed.gz", separator="\t", has_header=False, new_columns=["chr","start","end","gene_id", "?", "orientation"]
				  ).select(pl.exclude("?"))
targets_file = cd+"mpra_capture_flow/resources/target_regions.bed"
digest_file = cd+"mpra_capture_flow/resources/Digest_hg38_DpnII.txt"
digest = pl.read_csv(digest_file, skip_rows=1, separator="\t").select(~cs.matches("RE1|Restr"))
baits = pl.read_csv(cd+"mpra_capture_flow/resources/Arnould_3X_Mod_1_Covered.bed", separator="\t", has_header=False, new_columns=["chr","start","end","probe"])
nr_reads = pl.read_csv(cd+"mpra_capture_flow/results/CMPRA5/binned/"+config+"_readsperbin_OA.tsv", separator="\t")
#alu_repeats = pl.read_csv("../../data/repeatMasker_annotations/hg38_2020_rmsk_Alus.bed.gz", separator="\t", has_header=False, new_columns=["chr","start","end","type", "score", "strand"])
#short_enhancers = pl.read_csv("../../resources/short_enhancers.bed.gz", separator="\t", has_header=False)
#ccres = pl.read_csv("../../resources/GRCh38-cCREs.bed", separator="\t", has_header=False, new_columns=["chr","start","end","?", "enhancer_id", "description"])

## Add features 

### Nr of reads per bin

In [31]:
with_nr_reads = mpralm.with_columns(pl.when(pl.col("bin").str.ends_with("::")).then(pl.col("bin") + "null-null-null-null").otherwise(pl.col("bin")))\
	.join(nr_reads.with_columns(pl.when(pl.col("bin").str.ends_with("::")).then(pl.col("bin") + "null-null-null-null").otherwise(pl.col("bin"))), on="bin")
with_nr_reads = with_nr_reads.with_columns(pl.col("bin").str.split("::").list.to_struct()).unnest("bin").rename({
		"field_0": "left_bin", 
		"field_1": "right_bin"
		}) #.cast({cs.matches(".*_start|.*_end"): pl.Int32}, strict=False)
bins = pl.concat(with_nr_reads.select(cs.matches("_bin"))).str.splitn("-",4).struct.unnest().rename({
		"field_0": "chr", 
		"field_1": "start", 
		"field_2": "end",
		"field_3": "strand"
		}).select("chr", "start", "end", pl.lit(".").alias("name"), pl.lit(".").alias("score"), "strand") \
	.cast({cs.matches("start|end"): pl.Int32}, strict=False).filter(pl.col("chr").str.contains("chr")).sort("chr", "start", "end").unique()
bins.write_csv(cd+"resources/temp.single_bins.tsv", separator="\t", include_header=False)
bin_file = cd+"resources/temp.single_bins.tsv"

### TSS locations

In [32]:
!bedtools intersect -b $tss_file -a  $bin_file -wa | sort -k 1,1 -k2,2n | cut -f -3,6 | uniq > $cd"resources/temp.tss_bins.tsv"
#awk '{print $1"-"$4}' 
#awk 'print $ | uniq > ../../resources/temp.tss_bins.tsv


In [33]:
tss_bins = pl.read_csv(cd+"resources/temp.tss_bins.tsv", has_header=False, new_columns=["chr", "start", "end", "strand"], separator="\t").with_columns(tss=pl.lit("yes"))
tss_bins = tss_bins.select("tss", bin=pl.concat_str(["chr",  "start", "end", "strand"], separator="-"))

In [34]:
tss_data = with_nr_reads.join(tss_bins.rename({"tss": "tss_left"}), left_on="left_bin", right_on="bin",  how="left"
		  ).unique().join(tss_bins.rename({"tss": "tss_right"}), left_on="right_bin", right_on="bin",  how="left"
		  ).unique()
tss_data = tss_data.with_columns(cs.matches("tss").fill_null("no"))
tss_data = tss_data.with_columns(pl.when(pl.any_horizontal(cs.matches("tss") == "yes")).then(pl.lit("yes")).otherwise(pl.lit("no")).alias("any_tss"))

### Alu elements

Nevermind those...


In [8]:
!bedtools intersect -b $alu_file -a  $bin_file -wa | sort -k 1,1 -k2,2n | cut -f -3,6 |  uniq > $cd"resources/temp.alu_bins.tsv"

Error: Unable to open file /data/cephfs-2/unmirrored/groups/kircher/MPRA/CaptureCMPRA/data/repeatMasker_annotations/hg38_2020_rmsk_Alus.bed.gz. Exiting.


In [9]:
alu_bins = pl.read_csv("../../resources/temp.alu_bins.tsv", has_header=False, new_columns=["chr", "start", "end", "strand"], separator="\t").with_columns(alu=pl.lit("yes"))
alu_bins = alu_bins.select("alu", bin=pl.concat_str(["chr",  "start", "end", "strand"], separator="-"))

In [10]:
alu_data = tss_data.join(alu_bins.rename({"alu": "alu_left"}), left_on="left_bin", right_on="bin",  how="left"
		  ).unique().join(alu_bins.rename({"alu": "alu_right"}), left_on="right_bin", right_on="bin",  how="left"
		  ).unique()
alu_data = alu_data.with_columns(cs.matches("alu").fill_null("no"))
alu_data = alu_data.with_columns(pl.when(pl.any_horizontal(cs.matches("alu") == "yes")).then(pl.lit("yes")).otherwise(pl.lit("no")).alias("alu_overlap"))

### Encode CCREs

In [35]:
!bedtools intersect -b $ccres_file -a $bin_file -wo | \
cut -f -3,6,11-12  > $cd"resources/temp.ccres_bins.tsv" # | sort -k1,1 -k2,2n| bedtools merge -i stdin -c 4,5 -o distinct

In [36]:
ccres_bins = pl.read_csv(cd+"resources/temp.ccres_bins.tsv", has_header=False, new_columns=["chr", "start", "end", "strand", "eh_id", "description"], separator="\t")
ccres_bins = ccres_bins.select("eh_id", "description", bin=pl.concat_str(["chr", "start", "end", "strand"], separator="-")).group_by("bin").agg("eh_id", "description"
								).with_columns(eh_id = pl.col("eh_id").list.unique().list.join(","), description = pl.col("description").list.unique().list.join(","))

In [37]:
data_ccres = tss_data.join(ccres_bins.rename({"description": "screen_left"}), left_on="left_bin", right_on="bin",  how="left"
		  ).unique().join(ccres_bins.rename({"description": "screen_right"}), left_on="right_bin", right_on="bin",  how="left"
		  ).unique()
#data_ccres = data_ccres.with_columns(cs.matches("screen").fill_null("none"))
data_ccres.height

210123

In [38]:
data_ccres.filter(cs.matches("screen_left").is_null()).height

91197

### Target regions and their labels

In [39]:
!bedtools intersect -b <(tail -n +2 $targets_file) -a $bin_file -s -wo | cut -f -3,6,10,13  > $cd"resources/temp.target_bins.tsv" # | sort -k1,1 -k2,2n | bedtools merge -i stdin -c 4,5 -o distinct

In [40]:
target_bins = pl.read_csv(cd+"resources/temp.target_bins.tsv", has_header=False, new_columns=["chr", "start", "end", "strand", "target_gene", "label"], separator="\t")
target_bins = target_bins.select("target_gene", bin=pl.concat_str(["chr",  "start", "end", "strand"], separator="-"), targeted="label").group_by("bin").agg(
	"targeted", "target_gene").with_columns(
		pl.col('targeted').list.unique().list.join(","), 
		pl.col('target_gene').list.unique().list.join(","))
data_labeled = data_ccres.join(target_bins.rename({"targeted": "targeted_left", "target_gene": "target_gene_left"}), left_on="left_bin", right_on="bin",  how="left"
		  ).join(target_bins.rename({"targeted": "targeted_right", "target_gene": "target_gene_right"}), left_on="right_bin", right_on="bin",  how="left"
		  )
data_labeled = data_labeled.with_columns(cs.matches("target").fill_null("unlabeled"))
data_labeled.height

210123

#### Remove wrong direction promoters

In [41]:
data_labeled = data_labeled.filter(((pl.col("targeted_left") == "unlabeled") & (pl.col("left_bin").str.contains("-[+-]"))).not_())
data_labeled = data_labeled.filter(((pl.col("targeted_right") == "unlabeled") & (pl.col("right_bin").str.contains("-[+-]"))).not_())

In [ ]:
data_labeled.filter(pl.col("right_bin") == "chr2-9423172-9423811-+")

In [43]:
data_labeled = data_labeled.with_columns(pl.when(pl.all_horizontal(cs.matches("targeted").str.contains("positive"))).then(pl.lit("positive - positive"))
						 .when(pl.all_horizontal(cs.matches("targeted").str.contains("negative"))).then(pl.lit("negative - negative"))
						 .when(pl.all_horizontal(cs.matches("targeted").str.contains("target"))).then(pl.lit("target - target"))
						 .when(pl.all_horizontal(cs.matches("targeted").str.contains("unlabeled"))).then(pl.lit("other - other"))
						 .when(pl.any_horizontal(cs.matches("targeted").str.contains("positive")) & pl.any_horizontal(cs.matches("targeted").str.contains("negative"))).then(pl.lit("positive - negative"))
						 .when(pl.any_horizontal(cs.matches("targeted").str.contains("positive")) & pl.any_horizontal(cs.matches("targeted").str.contains("target"))).then(pl.lit("positive - target"))
						 .when(pl.any_horizontal(cs.matches("targeted").str.contains("negative")) & pl.any_horizontal(cs.matches("targeted").str.contains("target"))).then(pl.lit("negative - target"))
						 .when(pl.any_horizontal(cs.matches("targeted").str.contains("unlabeled")) & pl.any_horizontal(cs.matches("targeted").str.contains("target"))).then(pl.lit("target - other"))
						 .when(pl.any_horizontal(cs.matches("targeted").str.contains("unlabeled")) & pl.any_horizontal(cs.matches("targeted").str.contains("positive"))).then(pl.lit("positive - other"))
						 .when(pl.any_horizontal(cs.matches("targeted").str.contains("unlabeled")) & pl.any_horizontal(cs.matches("targeted").str.contains("negative"))).then(pl.lit("negative - other"))
						 .alias("label"))

In [44]:
data = data_labeled.with_columns(pl.when(pl.all_horizontal(cs.matches("screen").str.contains("PLS"))).then(pl.lit("PLS - PLS"))
						 .when(pl.any_horizontal(cs.matches("screen").str.contains("ELS")) & pl.any_horizontal(cs.matches("screen").str.contains("PLS"))).then(pl.lit("PLS - ELS"))
						 .when(pl.all_horizontal(cs.matches("screen").str.contains("ELS"))).then(pl.lit("ELS - ELS"))
						 .when(pl.all_horizontal(pl.any_horizontal(cs.matches("screen").str.contains("PLS")) & pl.any_horizontal(cs.matches("screen").str.contains("ELS|PLS").not_())))
			 			.then(pl.lit("PLS - undefined"))
						.when(pl.all_horizontal(pl.any_horizontal(cs.matches("screen").str.contains("ELS")) & pl.any_horizontal(cs.matches("screen").str.contains("ELS|PLS").not_())))
						.then(pl.lit("ELS - undefined"))
						 .when(pl.all_horizontal(cs.matches("screen").str.contains("PLS|ELS").not_())).then(pl.lit("undefined"))
						 .alias("interaction"))


In [45]:
data = data.select("logFC", "adj.P.Val", "P.Value", "left_bin", "right_bin", "nr_reads", "nr_seqs", "any_tss",
"screen_left", "screen_right", "targeted_left", "targeted_right", "target_gene_left", "target_gene_right", "label", "interaction")

### Distance between prom and enh

In [46]:
plotting_data = data.filter(pl.col("right_bin").str.contains("null").not_())
le20mb = plotting_data.with_columns(chr1 = pl.col("left_bin").str.split("-").list.get(0),
				   start1 = pl.col("left_bin").str.split("-").list.get(1).cast(pl.Int32),
				   end1 = pl.col("left_bin").str.split("-").list.get(2).cast(pl.Int32),
				   chr2 = pl.col("right_bin").str.split("-").list.get(0),
				   start2 = pl.col("right_bin").str.split("-").list.get(1).cast(pl.Int32),
				   end2 = pl.col("right_bin").str.split("-").list.get(2).cast(pl.Int32)).with_columns( 
				   dist = np.abs((pl.col("end1")+pl.col("start1"))/2 - (pl.col("end2") + pl.col("start2"))/2))
le20mb = le20mb.filter((pl.col("chr1") == pl.col("chr2")) & 
		       (pl.col("dist") <= 2000000))
le20mb = le20mb.select(~cs.matches("1|2"))
data = pl.concat([le20mb,
				 data.filter(pl.col("right_bin").str.contains("null")).with_columns(dist = None)
				 ])

### Activity relative to promoter only

In [47]:
data = data.with_columns(
	target_genes = pl.concat_str(pl.col("target_gene_left"), pl.col("target_gene_right"),
	separator=",").str.replace("unlabeled,", "").str.replace(",unlabeled", ""))

baited_interactions = data.filter(pl.col("label") != "other - other").filter(pl.col("label").str.contains("other"))

#baited_interactions = baited_interactions.select(pl.exclude("target_gene_left", "target_gene_right"))
baited_interactions = baited_interactions.filter(pl.col("target_genes").str.contains(",").not_())

In [48]:
# giving promoter and other end labels.
baited_interactions = baited_interactions.with_columns(
	pl.when(pl.col("targeted_left") == "unlabeled")
	.then(pl.col("right_bin")).otherwise(pl.col("left_bin")).alias("promoter"), 
	pl.when(pl.col("targeted_left") == "unlabeled")
	.then(pl.col("left_bin")).otherwise(pl.col("right_bin")).alias("OE"))

In [49]:
prom_only_logfcs = baited_interactions.filter(
	pl.col("right_bin").str.contains("null")).select("promoter", promoter_only = "logFC")
prom_only_logfcs = prom_only_logfcs.group_by("promoter").median()
relative_activity = baited_interactions.join(prom_only_logfcs, on="promoter").filter(pl.col("right_bin").str.contains("null").not_())
#relative_activity = relative_activity.with_columns(std = pl.col("logFC").std().over("promoter"))
relative_activity = relative_activity.with_columns(std = np.sqrt((pl.col("logFC")- pl.col("promoter_only"))
																 .pow(2).sum().over("promoter")/(pl.col("OE").count().over("promoter")-1)))

In [51]:
prom_only_logfcs.filter(pl.col("promoter") == "chr12-40224214-40224566-+")

promoter,promoter_only
str,f64


In [52]:
sign_changes = relative_activity.with_columns(
	pl.when(pl.col("logFC") > pl.col("promoter_only") + 2*pl.col("std"))
	.then(pl.lit("upregulating"))
	.when(pl.col("logFC") < pl.col("promoter_only") - 2*pl.col("std"))
	.then(pl.lit("downregulating"))
	.otherwise(pl.lit("no effect")).alias("effect"), 
	z_score = (pl.col("logFC") - pl.col("promoter_only"))/pl.col("std"))

In [53]:
final_data = data.join(sign_changes.select("left_bin", "right_bin", "effect", 'promoter', 'OE', 'promoter_only', 'std', "z_score"), 
				 on=['left_bin', 'right_bin'], how="left")


In [54]:
print(sign_changes.columns)
print(data.columns)

['logFC', 'adj.P.Val', 'P.Value', 'left_bin', 'right_bin', 'nr_reads', 'nr_seqs', 'any_tss', 'screen_left', 'screen_right', 'targeted_left', 'targeted_right', 'target_gene_left', 'target_gene_right', 'label', 'interaction', 'dist', 'target_genes', 'promoter', 'OE', 'promoter_only', 'std', 'effect', 'z_score']
['logFC', 'adj.P.Val', 'P.Value', 'left_bin', 'right_bin', 'nr_reads', 'nr_seqs', 'any_tss', 'screen_left', 'screen_right', 'targeted_left', 'targeted_right', 'target_gene_left', 'target_gene_right', 'label', 'interaction', 'dist', 'target_genes']


## Write to file

In [55]:
config = "promoteroa_binaware" #let's correct the typo

In [56]:
final_data.write_csv(cd+"results/MPRA_analysis/CMPRA5/labeled_data_"+config+"_OA.tsv", separator="\t")

In [57]:
final_data.height

87087

In [117]:
data.height  == data.n_unique(["left_bin", "right_bin"])

True